# Financial Accounting ETL

This notebook transforms the [Online Retail](https://archive.ics.uci.edu/dataset/352/online+retail) dataset into a double-entry compliant bookkeeping format. Realistically, financial data would come from an ERP system (Workday, SAP) in this exact format.

In [10]:
import pandas as pd

In [22]:
online_schema = {
    'InvoiceNo': 'string',
    'InvoiceDate': 'string',
    'StockCode': 'string',
    'Quantity': 'int64',
    'UnitPrice' : 'float64',
    'CustomerID' : 'string',
    'Country' : 'string'
}
schema_de = {
    'Date': 'string',
    'Account': 'string',
    'Debit': 'float64',
    'Credit': 'float64',
    'Category': 'string',
    'Transaction Type': 'string',
    'Customer_Vendor': 'string',
    'Payment Method': 'string',
    'Country': 'string'
}

In [23]:
online = pd.read_csv('../data/online_retail.csv', parse_dates=True, dtype=online_schema).sample(100000)
online.to_csv('../data/online_retail_sampled.csv', index=False)
online['InvoiceDate'] = pd.to_datetime(pd.to_datetime(online['InvoiceDate']).dt.date)
online['PurchaseTotal'] = online['Quantity']*online['UnitPrice']
online['InvoiceDate'] = online['InvoiceDate'] + pd.to_timedelta(52*11, unit='W')
online.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,PurchaseTotal
211295,555359,21974,SET OF 36 PAISLEY FLOWER DOILIES,1,2022-05-19,1.45,18135.0,United Kingdom,1.45
506613,579146,21479,WHITE SKULL HOT WATER BOTTLE,3,2022-11-14,4.25,16347.0,United Kingdom,12.75
203025,554511,35004B,SET OF 3 BLACK FLYING DUCKS,1,2022-05-10,5.79,<NA>,United Kingdom,5.79
344076,566981,85099C,JUMBO BAG BAROQUE BLACK WHITE,10,2022-09-02,2.08,15498.0,United Kingdom,20.80
274567,560926,21327,SKULLS WRITING SET,1,2022-07-08,0.83,<NA>,United Kingdom,0.83


# Enforcing Double-Entry Bookkeeping Compliant Format

This dataset is simply formatted. In order to transform to Double-Entry, we must duplicate each row and specify debited and credited accounts so that our accounting equation is balanced.

In [24]:
online_cleaned = online.loc[(online['Description'].notna())&(online['Quantity'].notna())&(online['UnitPrice'].notna()) \
&(online['CustomerID'].notna())&(online['Country'].notna())]

double_entry = pd.DataFrame(columns=schema_de.keys()).astype(schema_de)

In [25]:
def create_double_entry(row):
    debit_row = {
        'Date': row['InvoiceDate'],
        'Account': 'Accounts Receivable',
        'Debit': row['PurchaseTotal'],
        'Credit': 0,
        'Category': 'Revenue',
        'Transaction Type': 'Sale',
        'Customer_Vendor': row['CustomerID'],
        'Payment Method': 'Credit Card',
        'Country': row['Country']
    }

    credit_row = {
        'Date': row['InvoiceDate'],
        'Account': 'Sales Revenue',
        'Debit': 0,
        'Credit': row['PurchaseTotal'],
        'Category': 'Revenue',
        'Transaction Type': 'Sale',
        'Customer_Vendor': row['CustomerID'],
        'Payment Method': 'Credit Card',
        'Country': row['Country']
    }
    return pd.DataFrame([debit_row, credit_row], columns=schema_de.keys())

double_entry = pd.concat(online_cleaned.apply(create_double_entry, axis=1).to_list(), ignore_index=True)
double_entry.to_csv('../data/double_entry_online_retail.csv', index=False)